In [81]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'
%matplotlib inline
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')


from sklearn.metrics import make_scorer

# 시계열

from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import statsmodels.api as sm


from sklearn.metrics import mean_absolute_error, mean_squared_error

data_dir='C:/Users/kdh2/BIGCON/data/total'
data_list = os.listdir(data_dir)


#항목별 저장
data_list_single_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인타자')]
data_single_hitter = [pd.read_csv(os.path.join(data_dir, data_list_single_hitter[x]), encoding='cp949') for x in range(len(data_list_single_hitter))]
data_list_single_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인투수')]
data_single_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_single_pitcher[x]), encoding='cp949') for x in range(len(data_list_single_pitcher))]
data_list_games = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('경기')]
data_games = [pd.read_csv(os.path.join(data_dir, data_list_games[x]), encoding='cp949') for x in range(len(data_list_games))]
data_list_player_enroll = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('등록선수')]
data_player_enroll = [pd.read_csv(os.path.join(data_dir, data_list_player_enroll[x]), encoding='cp949') for x in range(len(data_list_player_enroll))]
data_list_players = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('선수')]
data_players = [pd.read_csv(os.path.join(data_dir, data_list_players[x]), encoding='cp949') for x in range(len(data_list_players))]
data_list_teams = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀')]
data_teams = [pd.read_csv(os.path.join(data_dir, data_list_teams[x]), encoding='cp949') for x in range(len(data_list_teams))]
data_list_team_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀타자')]
data_team_hitter = [pd.read_csv(os.path.join(data_dir, data_list_team_hitter[x]), encoding='cp949') for x in range(len(data_list_team_hitter))]
data_list_team_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀투수')]
data_team_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_team_pitcher[x]), encoding='cp949') for x in range(len(data_list_team_pitcher))]


team_hitter=pd.concat(data_team_hitter,axis=0)
team_pitcher=pd.concat(data_team_pitcher,axis=0)
for i in range(0,5):
     data_team_hitter[i]['AVG']=data_team_hitter[i]['HIT']/data_team_hitter[i]['AB']

team.columns

from datetime import datetime as dt
from dateutil.parser import parse
team_2016=data_team_hitter[0]
team_2017=data_team_hitter[1]
team_2018=data_team_hitter[2]
team_2019=data_team_hitter[3]
team_2020=data_team_hitter[4]
df_list=[team,team_2016,team_2017,team_2018,team_2019,team_2020]
for i in df_list:
    i['y']=i['HIT']/i['AB']
    
    i['ds']=pd.to_datetime(i['G_ID'].apply(lambda x: parse(str(x[:8])+str(x[12:])+'00000',yearfirst=True)),errors='raise')
    i['H1']=i['HIT']-i['H2']-i['H3']-i['HR']
    


## 시계열 모델 사용 조건: 정상성 만족
- 이에 따라 팀 투수, 팀 타자 데이터에 대해 adfuller 정상성 검정 시행

## 결과
- 타자 데이터의 KK변수 (삼진) 외엔 모두 정상성을 만족함.
- 이 또한 유의수준 0.05하에서이며, p값이 약 0.06 정도로 유의수준에 따라 정상성을 만족한다고 볼 수도 있음
- 따라서 시계열 모델을 사용할 것이며, task가 예측인 만큼 단변량 시계열 모델 외에 정확도가 높은 VAR model을 사용

## 한계
- VAR은 제한된 수의 변수만을 사용한다는 점에서 변수 선정에 의한 편의가 발생
- 또한 lag와 변수를 어떻게 주냐에 따라 결과값이 달라질 수 있음


## 해결방안
- LGBM, LSTM과 같은 시계열 모델과 독립적인 모델과의 앙상블을 통해 Model Variance 줄이기
- 데이터의 수가 적음을 고려하여, 3~4개 정도의 주요 변수만 선정
    => grangercausality 인과관계 검정을 통해 가장 타깃 변수에 유의한 영향을 미치는 변수만 선정
- lag는 AIC를 기준으로 선정하되, 10이 넘지 않도록 설정

In [82]:
from statsmodels.tsa.stattools import adfuller

def print_adfuller(inputSeries):
    result = adfuller(inputSeries)
    print('p-value: %f' % result[1])
    return result[1]

In [80]:

t=list(team_hitter['T_ID'].unique())
hit_team_list=[0]*10
for i in range(len(hit_team_list)):
    hit_team_list[i]=team_hitter[team_hitter['T_ID']==t[i]]

In [83]:

t=list(team_pitcher['T_ID'].unique())
pit_team_list=[0]*10
for i in range(len(pit_team_list)):
    pit_team_list[i]=team_pitcher[team_pitcher['T_ID']==t[i]]

In [85]:
#타자 데이터의 모든 팀별 데이터의 모든 컬럼에 대해 정상성 검정 시행
hit_con_cols=team_hitter.columns[6:]



results=[]
for t in hit_team_list:

    for i in hit_con_cols:
        
        
        pvalue=print_adfuller(t[i])
        if pvalue>0.05:
            print(i)
        results.append(pvalue)

p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.001153
p-value: 0.000009
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0

In [94]:
pit_con_cols

Index(['CG_CK', 'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK',
       'ERR', 'R', 'ER', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT'],
      dtype='object')

In [103]:
#투수 데이터의 모든 팀별 데이터의 모든 컬럼에 대해 정상성 검정 시행
pit_con_cols=team_pitcher.columns[6:]

pit_con_cols=pit_con_cols.drop('WLS')

results=[]
for t in pit_team_list:

    for i in pit_con_cols:
        
        
        pvalue=print_adfuller(t[i])
        if pvalue>0.05:
            print(i)
        results.append(pvalue)

p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000043
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000139
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0.000000
p-value: 0